### I started this kernel a while ago with no end game in mind so now I think I will just take a look at mortality rates in PA and the U.S. and add some visualizations.

In [96]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [97]:
df = pd.read_csv('../input/mort.csv')
df.head()

In [98]:
df.info()

In [99]:
df.describe()

There is a lot to this data.

- Mortality Rates including Min and Max

- Category of _Mortality_

- Location at State and County Level

- Percent Change from 1980 to end

### Percent Change in Mortality Rates
I would like to take a gander at these first.

In [100]:
pc_cols = ['Location','FIPS','Category',
           '% Change in Mortality Rate, 1980-2014',
           '% Change in Mortality Rate, 1980-2014 (Min)',
           '% Change in Mortality Rate, 1980-2014 (Max)']
pct_chg = df[pc_cols].set_index('Location')
pct_chg.head()

In [101]:
pa = pct_chg.loc['Pennsylvania']
pa

In [102]:
pa_by_cat = pa.reset_index()
pa_by_cat.drop(['Location','FIPS'],axis=1,inplace=True)
pa_by_cat.set_index(['Category'],inplace=True)
pa_by_cat['% Change in Mortality Rate, 1980-2014'].plot(kind='bar', rot=90, figsize=(14,8));
plt.ylabel('% Change in Mort Rate, 1980-2014');

Oh wow. There is a lot to digest just in this chart alone.  The majority of the categories appear to be going down for Pennsylvania residents.

Lets take a look at just the positive changes.

In [103]:
pa_by_cat[pa_by_cat['% Change in Mortality Rate, 1980-2014'] > 0]

The clear elephant here is Mental and Substance Use Disorders. I would tend to think that this is stemming from the information of these diseases being more prevalent in society over this 34 year period.

Lets cut the data set down furth and focuz on these ones in PA where the pct change was greater than 30% and compare them amongst the US as a whole.

In [115]:
pa_cats_over_30 = ['HIV/AIDS and tuberculosis', 'Neurological disorders',
                   'Mental and substance use disorders', 'Maternal disorders']

mort_rates = ['Location','Category','Mortality Rate, 1980*','Mortality Rate, 1985*','Mortality Rate, 1990*','Mortality Rate, 1995*',
              'Mortality Rate, 2000*','Mortality Rate, 2005*','Mortality Rate, 2010*','Mortality Rate, 2014*']

locs = ['United States','Pennsylvania']

In [165]:
pa_us = df[mort_rates].set_index(['Category'])
pa_us_cat = pa_us.loc[pa_cats_over_30]
pa_us_df = pa_us_cat[(pa_us_cat.Location == locs[1]) | (pa_us_cat.Location == locs[0])]
pa_us_df.reset_index(inplace=True)
pa_us_df

Here we have our comparison data frame I was looking for. After much trial and error I think I know how we can get this working the way I want it to visualize it. 

First we need to change our mortality rate columns to a pd.datetime index.

In [251]:
temp2 = pa_us_df.melt(id_vars=mort_rates[:2], value_vars=mort_rates[2:])
temp2.head()

In [272]:
# g=sns.factorplot(y='value', x='Location', hue='Category', 
#                  col='variable', col_wrap=4,
#                  data=temp2, kind='bar')

g=sns.factorplot(y='value', x='variable', col='Location', hue='Category',
                 data=temp2, aspect=2, size=5)
g.set_xticklabels(rotation=90)

There is a clear trend for __Neuroligical__ and __Mental and Substance Use__ disorders over the years. 

The __Maternal__ disorders dont appear to have changed all that much (which is a good thing) and thankfully we have made significant headway in __HIV/AIDS__.

Pennsylvania seems to follow the U.S. as a whole trend wise very closely.


In [266]:
pcmr = ['Location','Category','% Change in Mortality Rate, 1980-2014']
pa_us_pc = df[pcmr].set_index(['Category'])
pa_us_pc_cat = pa_us_pc.loc[pa_cats_over_30]
pa_us_pc_df = pa_us_pc_cat[(pa_us_pc_cat.Location == locs[1]) | (pa_us_pc_cat.Location == locs[0])]
pa_us_pc_df.reset_index(inplace=True)
pa_us_pc_df

In [277]:
g2=sns.factorplot(y='% Change in Mortality Rate, 1980-2014', 
                  x='Location', hue='Category', 
                  size=5, aspect=3,
                  data=pa_us_pc_df, kind='bar')

As a percent change over the years, Pennsylvania has seen a significant increase in death by __Mental and Substance__ use disorders. Its tough to tell where this stems from but it is possible it could be:

- An actual change from increased substance use or
- A change in categorization as the times have made more information available to be able to classify deaths like this